In [0]:
%pip install pydicom==3.0.1

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import logging
logger = logging.getLogger("zipdcm")
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(filename)s:%(lineno)s - %(levelname)s - %(message)s')
# Create a handler (e.g., StreamHandler for console output)
handler = logging.StreamHandler()
handler.setFormatter(formatter)

# Add the handler to the logger
logger.addHandler(handler)

### How many zip files 
do we have on `/Volumes/hls_radiology/tcia/downloads/tciaDownload`

In [0]:
# how many zips do we have?
df = (
    spark.read.format("binaryFile")
        .option("pathGlobFilter", "*.zip")
        .option("recursiveFileLookup", "true")
        .load("/Volumes/hls_radiology/tcia/downloads/tciaDownload").drop('content')
    )
df.count()

1416

In [0]:
%sql
drop table douglas_moore.pixels.zipdcm_test;

In [0]:
from zip_dcm_ds import ZipDCMDataSource

import time
start_time = time.time()
total_worker_cores = 16 


# Add our custom Python DataSource for DICMO files storaged in a Zip Archive
spark.dataSource.register(ZipDCMDataSource)

def test_scale(spark, numPartitions=64):
    df = (
        spark.read
          .option('numPartitions',f'{numPartitions}')
          .format("zipdcm")
          .load("/Volumes/hls_radiology/tcia/downloads/tciaDownload") #1416 zip files
    )
    df.write.format('delta').mode('overwrite').option("mergeSchema", "true").saveAsTable("douglas_moore.pixels.zipdcm_test")

test_scale(spark)


end_time = time.time()
operation_time = end_time - start_time
logger.info(f"Operation time: {operation_time} seconds and {operation_time * total_worker_cores} core-seconds")

2025-05-26 19:16:48,195 - command-2588019255111356-4200483778:25 - INFO - Operation time: 242.62723803520203 seconds and 3882.0358085632324 core-seconds
2025-05-26 19:16:48,195 - command-2588019255111356-4200483778:25 - INFO - Operation time: 242.62723803520203 seconds and 3882.0358085632324 core-seconds
2025-05-26 19:16:48,195 - command-2588019255111356-4200483778:25 - INFO - Operation time: 242.62723803520203 seconds and 3882.0358085632324 core-seconds


## Summary stats

In [0]:
%sql
select
count(distinct path) paths,
count(distinct split(path,'::')[0]) zipfiles,
count(distinct rowid) rowids, count(1) records
from douglas_moore.pixels.zipdcm_test

paths,zipfiles,rowids,records
107158,1416,8447,107158


In [0]:
%sql
select zipfile,count(distinct dcmfile) 
from douglas_moore.pixels.zipdcm_test
group by zipfile 
order by 2 desc

zipfile,count(DISTINCT dcmfile)
null,0


## Sample data records

In [0]:
%sql
select * 
from douglas_moore.pixels.zipdcm_test
limit 100;

rowid zipfile dcmfile meta path 309 null null {'00080008': {'vr': 'CS', 'Value': ['ORIGINAL', 'PRIMARY', '']}, '00080016': {'vr': 'UI', 'Value': ['1.2.840.10008.5.1.4.1.1.1.1']}, '00080018': {'vr': 'UI', 'Value': ['1.3.6.1.4.1.14519.5.2.1.8700.9920.285741939975147332449245627189']}, '00080020': {'vr': 'DA', 'Value': ['20380730']}, '00080021': {'vr': 'DA', 'Value': ['20380730']}, '00080022': {'vr': 'DA', 'Value': ['20380730']}, '00080023': {'vr': 'DA', 'Value': ['20380730']}, '00080030': {'vr': 'TM', 'Value': ['163605']}, '00080031': {'vr': 'TM', 'Value': ['163631']}, '00080032': {'vr': 'TM', 'Value': ['163723']}, '00080033': {'vr': 'TM', 'Value': ['163605']}, '00080050': {'vr': 'SH'}, '00080060': {'vr': 'CS', 'Value': ['DX']}, '00080068': {'vr': 'CS', 'Value': ['FOR PRESENTATION']}, '00080070': {'vr': 'LO', 'Value': ['Philips Medical Systems']}, '00080090': {'vr': 'PN'}, '00081030': {'vr': 'LO', 'Value': ['XR CHEST AP PORTABLE']}, '00081032': {'vr': 'SQ', 'Value': [{'00080100': {'vr': 'SH', 'Value': ['CHESTAPPAP']}, '00080102': {'vr': 'SH', 'Value': ['SECTRA']}, '00080104': {'vr': 'LO', 'Value': ['XR CHEST AP PORTABLE']}}]}, '0008103E': {'vr': 'LO', 'Value': ['AP']}, '00081090': {'vr': 'LO', 'Value': ['MobileDiagnost wDR']}, '00081111': {'vr': 'SQ', 'Value': [{'00081150': {'vr': 'UI', 'Value': ['1.2.840.10008.3.1.2.3.3']}, '00081155': {'vr': 'UI', 'Value': ['1.3.6.1.4.1.14519.5.2.1.8700.9920.216749826853087025430024514921']}}]}, '00083010': {'vr': 'UI', 'Value': ['1.3.6.1.4.1.14519.5.2.1.8700.9920.211127634248772974970651716291']}, '00100010': {'vr': 'PN', 'Value': [{'Alphabetic': 'MIDI_1_1_026'}]}, '00100020': {'vr': 'LO', 'Value': ['MIDI_1_1_026']}, '00100030': {'vr': 'DA'}, '00100040': {'vr': 'CS', 'Value': ['M']}, '00101010': {'vr': 'AS', 'Value': ['046Y']}, '00101020': {'vr': 'DS', 'Value': [1.632]}, '00101030': {'vr': 'DS', 'Value': [81.965]}, '00102000': {'vr': 'LO'}, '00102110': {'vr': 'LO'}, '00102160': {'vr': 'SH', 'Value': ['European']}, '001021B0': {'vr': 'LT'}, '001021C0': {'vr': 'US', 'Value': [4]}, '00120062': {'vr': 'CS', 'Value': ['YES']}, '00120063': {'vr': 'LO', 'Value': ['Per DICOM PS 3.15 AnnexE. Details in 0012,0064']}, '00120064': {'vr': 'SQ', 'Value': [{'00080100': {'vr': 'SH', 'Value': ['113100']}, '00080102': {'vr': 'SH', 'Value': ['DCM']}, '00080104': {'vr': 'LO', 'Value': ['Basic Application Confidentiality Profile']}}, {'00080100': {'vr': 'SH', 'Value': ['113101']}, '00080102': {'vr': 'SH', 'Value': ['DCM']}, '00080104': {'vr': 'LO', 'Value': ['Clean Pixel Data Option']}}, {'00080100': {'vr': 'SH', 'Value': ['113104']}, '00080102': {'vr': 'SH', 'Value': ['DCM']}, '00080104': {'vr': 'LO', 'Value': ['Clean Structured Content Option']}}, {'00080100': {'vr': 'SH', 'Value': ['113105']}, '00080102': {'vr': 'SH', 'Value': ['DCM']}, '00080104': {'vr': 'LO', 'Value': ['Clean Descriptors Option']}}, {'00080100': {'vr': 'SH', 'Value': ['113107']}, '00080102': {'vr': 'SH', 'Value': ['DCM']}, '00080104': {'vr': 'LO', 'Value': ['Retain Longitudinal Temporal Information Modified Dates Option']}}, {'00080100': {'vr': 'SH', 'Value': ['113108']}, '00080102': {'vr': 'SH', 'Value': ['DCM']}, '00080104': {'vr': 'LO', 'Value': ['Retain Patient Characteristics Option']}}, {'00080100': {'vr': 'SH', 'Value': ['113109']}, '00080102': {'vr': 'SH', 'Value': ['DCM']}, '00080104': {'vr': 'LO', 'Value': ['Retain Device Identity Option']}}, {'00080100': {'vr': 'SH', 'Value': ['113111']}, '00080102': {'vr': 'SH', 'Value': ['DCM']}, '00080104': {'vr': 'LO', 'Value': ['Retain Safe Private Option']}}]}, '00130010': {'vr': 'LO', 'Value': ['CTP']}, '00131010': {'vr': 'LO', 'Value': ['MIDI-B-Curated-Validation']}, '00131013': {'vr': 'LO', 'Value': ['29059806']}, '00131015': {'vr': 'LO', 'Value': ['1']}, '00180015': {'vr': 'CS', 'Value': ['CHEST']}, '00180060': {'vr': 'DS', 'Value': [90.0]}, '00181020': {'vr': 'LO', 'Value': ['2.1']}, '00181030': {'vr': 'LO', 'Value': ['Portable Chest']}, '00181050': {'vr': 'DS', 'Value': [0.148]}, '0018